# Config

In [39]:
# Set Variables
date_string = '20230903'
weekStart = 1
weekEnd = 2
extra_defensive_points = 18

In [3]:
# Import dependencies
import pandas as pd
import numpy as np
from sleeperpy import User, Leagues, Players
import json
from requests_html import HTMLSession


# Get Sleeper

In [4]:
def processRosters(league_id):
    # Get rosters for all league members
    rosters = pd.DataFrame(Leagues.get_rosters(league_id))
    # Explode the player columns to find how they are placed on the roster
    result = pd.DataFrame(columns=['owner_id', 'player_id'])
    for col in ['taxi', 'starters', 'players']:
        df = rosters[['owner_id', col]].explode(col)
        df = df.rename(columns={col:'player_id'})
        df[col] = True
        result = result.merge(df, how='outer', on=['owner_id', 'player_id'])
    return result

In [5]:
def procesOwners(league_id):
    owners = pd.DataFrame(Leagues.get_users(league_id))
    list = []
    for i in range(len(owners)):
        if 'team_name' in owners['metadata'][i]:
            list.append(owners['metadata'][i]['team_name'])
        else:
            list.append(owners['display_name'][i])
    owners['franchise_name'] = list
    owners = owners[['user_id', 'franchise_name', 'display_name']]
    return owners

In [6]:
def processPlayers():
    # Use requests_html to get players since sleeper-py seems to be broken here
    session = HTMLSession()
    r = session.get('https://api.sleeper.app/v1/players/nfl')
    df = pd.DataFrame(json.loads(r.html.html)).T
    df = df[[
        'player_id', 'full_name','birth_date','age','weight', 'height',
        'team','position','fantasy_positions','depth_chart_order','depth_chart_position',
        'status', 'injury_status','injury_notes','injury_start_date', 'practice_participation', 'practice_description',
        'years_exp',
        'active',
    ]]
    df = df.loc[df['position'].isin(['QB','RB','WR','TE','K','P','DEF'])]
    df.loc[df['position'].isin(['P', 'K']), 'position'] = 'PK'
    return df

In [7]:
# Get User's Sleeper ID
user_id = User.get_user('dirtywizard')['user_id']
# Get the User's Sleeper league ID
league_id = Leagues.get_all_leagues('725907041763303424', 'nfl', 2023)[0]['league_id']
# Get all players on rosters
rosters = processRosters(league_id)
# Get info on all owners
owners = procesOwners(league_id)
# Get info on all players
players = processPlayers()

# Get Fantasy Sharks

In [43]:
from requests_html import HTMLSession, AsyncHTMLSession
import pandas as pd
import time
session = AsyncHTMLSession()
sharks = pd.DataFrame()

# Loop through weeks and positions
posList = ['1','2','4','5','7','15','6']
weekList = [str(int(x)) for x in np.linspace(787, 804, 18)]
for posKey in posList:
    for weekKey in weekList[weekStart:weekEnd]:
        url = f'https://www.fantasysharks.com/apps/bert/forecasts/projections.php?League=&Position={posKey}&scoring=2&Segment={weekKey}&uid=4'
        # Scrape
        r = await session.get(url)
        await r.html.arender()
        element = r.html.find('#toolData')[0].html
        # Convert to df
        df = pd.read_html(element)[0]
        df = df.loc[df['#'].str.isnumeric()]
        df['Week'] = int(weekKey) - 786
        sharks = pd.concat([sharks, df], axis=0, ignore_index=True)
        time.sleep(3)
        
# Convert raw prediction values to numeric
colList = ['Att', 'Comp', 'Pass Yds', 'Pass TDs', 'Int', 'Sck', 'Rush', 'Rsh Yds', 'Rsh TDs', 'Fum', 'Tgt', \
    'RZ Tgt', 'Rec', 'Rec Yds', 'Rec TDs', 'Kick Ret Yds', 'XPM', 'XPA', 'FGM', 'FGA', '10-19 FGM', '20-29 FGM', \
        '30-39 FGM', '40-49 FGM', '50+ FGM', 'Miss', 'Yds Allowed', 'Pts Agn', 'Scks', 'DefTD', 'Safts']
for col in colList:
    sharks[col] = sharks[col].astype('float64')
    
# Clean up
sharks = sharks.rename(columns={'Week':'week'})
# Create ID: Remove Rookie "R": format first name last name
cond1 = sharks['Player'].str[-1]=="R"
sharks.loc[cond1, 'Player'] = sharks.loc[cond1, 'Player'].str[:-1]
sharks['lName'] = sharks['Player'].str.split(", ", expand=True)[0]
sharks['fName'] = sharks['Player'].str.split(", ", expand=True)[1]
sharks['id_sharks'] = sharks['fName']+" "+sharks['lName']
sharks['id_sharks'] = sharks['id_sharks'].str.replace(".", "")
sharks['id_sharks'] = sharks['id_sharks'].str.title()
sharks['id_sharks'] = [" ".join(x.split(" ")[:2]) for x in sharks['id_sharks']]
# Reduce
sharks = sharks[['id_sharks', 'week']+colList]


In [44]:
sharks


,id_sharks,week,Att,Comp,Pass Yds,Pass TDs,Int,Sck,Rush,Rsh Yds,...,20-29 FGM,30-39 FGM,40-49 FGM,50+ FGM,Miss,Yds Allowed,Pts Agn,Scks,DefTD,Safts
0,Josh Allen,2,34.2,21.6,245.0,2.0,1.0,1.9,8.3,50.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Patrick Mahomes,2,35.3,23.8,277.0,2.3,0.5,1.5,3.7,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Jalen Hurts,2,29.9,19.3,231.0,1.3,0.4,2.1,10.0,46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Joe Burrow,2,32.4,21.9,244.0,1.7,0.8,3.3,4.2,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Tua Tagovailoa,2,34.7,22.7,271.0,1.7,0.6,1.5,2.1,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,Tennessee Titans,2,NaN,NaN,NaN,NaN,0.8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,387.7,23.7,1.8,0.02,0.00
530,Atlanta Falcons,2,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,370.2,22.9,0.9,0.20,0.02
531,Seattle Seahawks,2,NaN,NaN,NaN,NaN,0.8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,451.1,30.1,1.9,0.05,0.00
532,Minnesota Vikings,2,NaN,NaN,NaN,NaN,0.7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,433.9,31.4,1.7,0.10,0.00


# Get Ourlads

In [ ]:
# Import dependencies
from requests_html import HTMLSession, AsyncHTMLSession
import pandas as pd
session = AsyncHTMLSession()

# Get page
r = await session.get("https://www.ourlads.com/nfldepthcharts/depthcharts.aspx")
await r.html.arender()

element = r.html.find('#ctl00_phContent_gvChart')[0].html

# Convert to df
df = pd.read_html(element)[0]
# Filter for only the needed columns
df = df[['Team', 'Pos', 'Player 1', 'Player 2', 'Player 3', 'Player 4', 'Player 5']]
# Rename columns of Position Ranks; limit number of ranks to three
df = df.rename(columns={
    'Player 1':'1',
    'Player 2':'2',
    'Player 3':'3',
    'Player 4':'3',
    'Player 5':'3',
})
# Filter only relevant positions
posList = ['LWR', 'RWR', 'SWR', 'TE', 'QB', 'RB', 'PK', 'PR', 'KR', 'RES']
df = df.loc[df['Pos'].isin(posList)]

# Transpose columns to rows to get position ranks in row form rather than column
df = df.melt(id_vars=["Team", "Pos"], 
    var_name="posRank", 
    value_name="playerName")
    # Rename columns to match MyFantasyLeague
df = df.rename(columns={'Team':'team', 'Pos':'pos'})
# Create id_ourlads column
df = df.dropna(subset='playerName')
df['lName'] = df['playerName'].str.split(", ", expand=True)[0]
df['fName'] = df['playerName'].str.split(", ", expand=True)[1].str.split(" ", expand=True)[0]
df['id_ourlads'] = df['fName'] + " " + df['lName']
df['id_ourlads'] = df['id_ourlads'].str.replace(".", "")
df['id_ourlads'] = df['id_ourlads'].str.title()
df['id_ourlads'] = [" ".join(x.split(" ")[:2]) for x in df['id_ourlads']]

# Highlight PR & KRs
cond1 = df['pos']=='PR'
cond2 = df['pos']=='KR'
cond3 = df['posRank']=="1"
prs = df.loc[cond1 & cond3]['id_ourlads'].unique()
krs = df.loc[cond2 & cond3]['id_ourlads'].unique()
df.loc[df['id_ourlads'].isin(prs), 'PR'] = True
df.loc[df['id_ourlads'].isin(krs), 'KR'] = True

# Reduce
df = df.loc[(df['PR']==True) | (df['KR']==True)]
df = df.drop_duplicates(subset='id_ourlads', ignore_index=True)
df = df[['id_ourlads', 'PR', 'KR']]

ourlads = df.copy()


In [9]:
# Calculate stats
ptsDict = {
'Att':0, 
'Comp':0, 
'Pass Yds':.04, 
'Pass TDs':4,
'Int':-2, 
'Sck':0, 
'Rush':.1, 
'Rsh Yds':.1,
'Rsh TDs':6, 
'Fum':-2, 
'Tgt':0, 
'RZ Tgt':0, 
'Rec':.25, 
'Rec Yds':.1, 
'Rec TDs':6, 
'Kick Ret Yds':.1, 
'XPM':1,
'XPA':0, 
'FGM':3, 
'FGA':0, 
'10-19 FGM':0, 
'20-29 FGM':0, 
'30-39 FGM':.5, 
'40-49 FGM':1.5,
'50+ FGM':2, 
'Miss':0, 
'Yds Allowed':-.027, 
'Pts Agn':-.29, 
'Scks':1, 
'DefTD':6, 
'Safts':2
}

# multiply by points multiplier
for col in ptsDict.keys():
    sharks[col] = sharks[col] * ptsDict[col]
# Add up point columns
sharks['pts_proj'] = sharks[ptsDict.keys()].sum(axis=1)
# Add extras for defense starting values
sharks.loc[sharks['Yds Allowed'].notna(), 'pts_proj'] = sharks.loc[sharks['Yds Allowed'].notna(), 'pts_proj'] + extra_defensive_points

# Create points dataset

In [35]:
# Left off here

In [36]:
sharks.head(1)

,id_sharks,week,pts_proj
0,Josh Allen,2,23.63


In [37]:
ourlads.head(1)

,id_ourlads,PR,KR
0,Greg Dortch,True,True


In [13]:
# Read dictionary of IDs
ids = pd.read_csv('data_sharks/id_dict.csv', dtype={'player_id':'str'})
ids = ids[['full_name', 'player_id', 'id_sharks', 'id_ourlads']]
ids

,full_name,player_id,id_sharks,id_ourlads
0,Aaron Bailey,4683,NaN,NaN
1,Aaron Burbridge,3368,NaN,NaN
2,Aaron Dobson,1365,NaN,NaN
3,Aaron Dykes,11548,NaN,NaN
4,Aaron Fuller,6967,NaN,NaN
...,...,...,...,...
3647,Zeke Vandenburgh,NaN,NaN,Zeke Vandenburgh
3648,Zimari Manning,7211,NaN,NaN
3649,Zion Bowens,11159,NaN,NaN
3650,Zonovan Knight,8122,NaN,NaN


In [21]:
# Merge in points
full = players \
    .merge(ids[['player_id', 'id_sharks', 'id_ourlads']], how='left', on='player_id') \
    .merge(sharks, how='left', on='id_sharks') \
    .merge(ourlads, how='left', on='id_ourlads') \
    .merge(rosters, how='left', on='player_id') \
    .merge(owners, how='left', left_on='owner_id', right_on='user_id')
full


,player_id,full_name,birth_date,age,weight,height,team,position,fantasy_positions,depth_chart_order,...,pts_proj,PR,KR,owner_id,taxi,starters,players,user_id,franchise_name,display_name
0,6744,Greg Ward,1995-07-12,28,190,71,PHI,WR,[WR],None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,918,Niles Paul,1989-08-09,32,242,73,None,TE,[TE],None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4910,Quincy McDuffie,1990-09-24,31,178,70,None,WR,[WR],None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,663,Benjamin Watson,1980-12-18,41,255,75,None,TE,[TE],None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5385,Alec Bloom,1996-04-03,25,253,78,None,TE,[TE],None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11420,11533,Brandon Aubrey,1995-03-14,28,208,75,DAL,PK,[K],1,...,9.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11421,8074,Matt Seybert,1997-05-02,24,252,76,None,TE,[TE],None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11422,1408,Le'Veon Bell,1992-02-18,29,225,73,TB,RB,[RB],None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11423,7926,Carl Tucker,1997-02-06,24,250,74,None,TE,[TE],None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# Adjust point calculation based on PR/KR
full.loc[full['PR']==True, 'pts_proj'] = full.loc[full['PR']==True, 'pts_proj'] + (1)
full.loc[full['KR']==True, 'pts_proj'] = full.loc[full['KR']==True, 'pts_proj'] + (3)

In [23]:
# Sum points across the season
totals = full.groupby('player_id')['pts_proj'].sum().reset_index()
unis = full.drop_duplicates(subset='player_id', ignore_index=True).drop(columns='pts_proj')
sums = unis.merge(totals, how='left', on='player_id')

In [24]:
# Calculate relative points
relDict = {
    'QB':24,
    'RB':48,
    'WR':48,
    'TE':24,
    'PK':24,
    'DEF':24
}
sums['pts_proj'] = sums['pts_proj'].fillna(0)
sums = sums.sort_values(by='pts_proj', ascending=False)
for pos in relDict:
    cond1 = sums['position']==pos
    floor = sums.loc[cond1, 'pts_proj'].reset_index(drop=True)[relDict[pos]]
    sums.loc[cond1, 'relpts_proj'] = sums.loc[cond1, 'pts_proj'] - floor

In [25]:
# Tidy Up
sums = sums[['player_id', 'franchise_name', 'relpts_proj', 'pts_proj', 'full_name', 'age', 'team', 'position','depth_chart_order', 
       'years_exp', 'weight', 'height','taxi', 'starters', 'players', 'active', 
       'fantasy_positions', 'depth_chart_position', 'PR', 'KR',
       'status', 'injury_status', 'injury_notes','injury_start_date', 'practice_participation', 'practice_description',
       ]]
sums = sums.sort_values(by='relpts_proj', ignore_index=True, ascending=False)
sums.loc[sums['position']=='DEF', 'full_name'] = sums.loc[sums['position']=='DEF', 'team']

# Waiver Report

In [26]:
# waiver report
minpts = sums.loc[(sums['franchise_name']=='Croccity Body Snatchers')&(sums['taxi'].isna())]['pts_proj'].min()
cond1 = sums['franchise_name']=='Croccity Body Snatchers'
cond2 = sums['franchise_name'].isna()
cond3 = sums['pts_proj']>=minpts
waivers = sums.loc[(cond1|cond2)&cond3].reset_index(drop=True)
waivers.to_csv(f'waivers_{date_string}.csv', index=False)

# Write to CSV

In [27]:
sums.to_csv(f'processed_{date_string}.csv', index=False)

In [28]:
# Todo:
# Print any sleeper players who are not already in the ids dataset
# Strip out final R from rookies in sharks
# Build Waiver report
# Build trade report
# RosterOptimizer
# Viz franchise points

In [29]:
# Starter report
sums.loc[sums['starters']==True].groupby('franchise_name')['relpts_proj'].sum().sort_values()

franchise_name
DangeRUSS Last Ride          86.8844
Comeback 👑’s                173.2645
kevinbash                   188.1524
Idk much about soccer       232.3899
2014champ                   248.5451
Count of Monte Christian    287.5669
GusTheBus                   410.7942
📜 Providence 🪬 Spirits 🥂    451.2059
Verdanks Vacqueros FFC      553.5242
Pretty Big Wieners          610.4773
Saskatoon Squatches         789.8972
Croccity Body Snatchers     860.2586
Name: relpts_proj, dtype: float64

In [30]:
sums.loc[sums['starters']==True].groupby('franchise_name')['pts_proj'].sum().sort_values()

franchise_name
DangeRUSS Last Ride          160.0080
Comeback 👑’s                1833.2381
kevinbash                   1925.8060
Idk much about soccer       2028.1385
2014champ                   2121.7737
Count of Monte Christian    2160.7955
GusTheBus                   2283.6228
📜 Providence 🪬 Spirits 🥂    2324.0345
Verdanks Vacqueros FFC      2426.5528
Pretty Big Wieners          2483.3059
Saskatoon Squatches         2582.2908
Croccity Body Snatchers     2733.0872
Name: pts_proj, dtype: float64

# Merge ids

In [31]:
# sharks = sharks.drop_duplicates(subset='id_sharks', ignore_index=True)
# sharks['full_name'] = sharks['id_sharks']

In [32]:
# ids = pd.read_csv('data_sharks/id_dict.csv', dtype={'player_id':'str', 'id_player_sharks':'str'})
# ids = ids[['player_id', 'id_ourlads', 'full_name']]
# ids

In [33]:
# allids = ids.merge(sharks, how='outer', on='full_name')
# allids

In [34]:
# allids.to_csv('id_dict2.csv', index=False)